In [ ]:
"""
Created on Wed Oct 11 09:44 2023

Check distribution of input variables

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
%matplotlib qt5

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/NN_plots/input_vars/'

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/' 
outputpath_CVinput = inputpath_data+'EXTRAPOLATED_ISFDRAFT_CHUNKS/'
norm_metrics_file = xr.open_dataset(outputpath_CVinput + 'metrics_norm_wholedataset.nc')
norm_metrics = norm_metrics_file.sel(norm_method='std').drop('norm_method')#.to_dataframe()

In [ ]:
outputpath_nn = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/EXTRAPOLATED_ISFDRAFT_CHUNKS/'
tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

li = []
n = 0
for tblock in tqdm(tblock_dim):
    if tblock in [5,8,11]:
        n=n+1
        
    for kisf in isf_dim:
        
        df = pd.read_csv(outputpath_nn + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+str(tblock).zfill(3)+'.csv',index_col=[0,1])
        df['time'] = df['time'] + n*100
        df = df.set_index('time',append=True)
        li.append(df)

concat_all = pd.concat(li)

In [ ]:
concat_ds = concat_all.to_xarray()

In [ ]:
input_list = ['dGL','dIF','corrected_isfdraft','bathy_metry','slope_bed_lon','slope_bed_lat','slope_ice_lon','slope_ice_lat','theta_in','salinity_in','T_mean', 'S_mean', 'T_std', 'S_std','melt_m_ice_per_y']
label_list = ['Distance GL [m]','Distance IF [m]',
              'Ice draft depth \n [m below sea-level]','Bathymetry \n [m below sea-level]',
              'Slope bed lon [rad]','Slope bed lat [rad]','Slope ice lon [rad]','Slope ice lat [rad]',
              'Temperature \n [degrees C]','Salinity [psu]','Temperature mean \n [degrees C]', 'Salinity mean [psu]', 'Temperature std \n [degrees C]', 'Salinity std [psu]',
              'Melt [m ice per y]']

In [ ]:
concat_ds_of_int = concat_ds[input_list]

In [ ]:
concat_ds_of_int_normed = (concat_ds_of_int - norm_metrics[input_list].sel(metric='mean_vars')) / norm_metrics[input_list].sel(metric='range_vars')

In [ ]:
var_tot_normed = concat_ds_of_int_normed.var()
var_time_normed = concat_ds_of_int_normed.var('time').mean()
var_loc_normed = concat_ds_of_int_normed.var(['x','y']).mean()


In [ ]:
var_time_normed + var_loc_normed

In [ ]:
var_tot = concat_ds_of_int.var()
var_time = concat_ds_of_int.var('time').mean()
var_loc = concat_ds_of_int.var(['x','y']).mean()

In [ ]:
prop_var_time = var_time/var_tot
prop_var_loc = var_loc/var_tot


In [ ]:
concat_ds_of_int_normed.var('time').mean()

In [ ]:
concat_ds_of_int_normed.var(['x','y']).mean()

In [ ]:
concat_ds_of_int_normed['salinity_in'].mean('time').plot()